In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s5e6/sample_submission.csv
/kaggle/input/playground-series-s5e6/train.csv
/kaggle/input/playground-series-s5e6/test.csv


In [2]:
import pandas as pd

In [3]:
import pandas as pd

train = pd.read_csv('/kaggle/input/playground-series-s5e6/train.csv')
test = pd.read_csv('/kaggle/input/playground-series-s5e6/test.csv')


In [4]:
train.head()

,id,Temparature,Humidity,Moisture,Soil Type,Crop Type,Nitrogen,Potassium,Phosphorous,Fertilizer Name
0,0,37,70,36,Clayey,Sugarcane,36,4,5,28-28
1,1,27,69,65,Sandy,Millets,30,6,18,28-28
2,2,29,63,32,Sandy,Millets,24,12,16,17-17-17
3,3,35,62,54,Sandy,Barley,39,12,4,10-26-26
4,4,35,58,43,Red,Paddy,37,2,16,DAP


In [5]:
test.head()

,id,Temparature,Humidity,Moisture,Soil Type,Crop Type,Nitrogen,Potassium,Phosphorous
0,750000,31,70,52,Sandy,Wheat,34,11,24
1,750001,27,62,45,Red,Sugarcane,30,14,15
2,750002,28,72,28,Clayey,Ground Nuts,14,15,4
3,750003,37,53,57,Black,Ground Nuts,18,17,36
4,750004,31,55,32,Red,Pulses,13,19,14


In [6]:
X = train.drop(columns = ['id','Fertilizer Name'])
y = train['Fertilizer Name']

In [7]:
X = train.drop(columns=['id', 'Fertilizer Name'])
y = train['Fertilizer Name']

X_test = test.drop(columns=['id'])



In [8]:
from sklearn.preprocessing import LabelEncoder

encoder_soil = LabelEncoder()
encoder_crop = LabelEncoder()
encoder_fertilizer = LabelEncoder()



In [9]:
X['Soil Type'] = encoder_soil.fit_transform(X['Soil Type'])
X['Crop Type'] = encoder_crop.fit_transform(X['Crop Type'])

X.head()

,Temparature,Humidity,Moisture,Soil Type,Crop Type,Nitrogen,Potassium,Phosphorous
0,37,70,36,1,8,36,4,5
1,27,69,65,4,4,30,6,18
2,29,63,32,4,4,24,12,16
3,35,62,54,4,0,39,12,4
4,35,58,43,3,6,37,2,16


In [10]:
y_encode = encoder_fertilizer.fit_transform(y)


y_encode[:5]

y.head()

0       28-28
1       28-28
2    17-17-17
3    10-26-26
4         DAP
Name: Fertilizer Name, dtype: object

In [11]:

X_test['Soil Type'] = encoder_soil.fit_transform(X_test['Soil Type'])
X_test['Crop Type'] = encoder_crop.fit_transform(X_test['Crop Type'])


X_test.head()

,Temparature,Humidity,Moisture,Soil Type,Crop Type,Nitrogen,Potassium,Phosphorous
0,31,70,52,4,10,34,11,24
1,27,62,45,3,8,30,14,15
2,28,72,28,1,2,14,15,4
3,37,53,57,0,2,18,17,36
4,31,55,32,3,7,13,19,14


In [12]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y_encode, test_size=0.2, random_state=42)



In [13]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)



RandomForestClassifier(random_state=42)

In [14]:

y_probs = model.predict_proba(X_test)


y_pred_encoded = model.predict(X_test)


y_pred_labels = encoder_fertilizer.inverse_transform(y_pred_encoded)

print(y_probs[:5])
print(y_pred_labels[:5])

[[0.06 0.08 0.12 0.1  0.19 0.26 0.19]
 [0.09 0.06 0.32 0.09 0.12 0.2  0.12]
 [0.17 0.05 0.1  0.25 0.08 0.14 0.21]
 [0.04 0.1  0.23 0.1  0.07 0.15 0.31]
 [0.14 0.09 0.2  0.31 0.12 0.06 0.08]]
['DAP' '17-17-17' '20-20' 'Urea' '20-20']


In [15]:
import numpy as np

top3 = np.argsort(y_probs, axis=1)[:, -3:][:, ::-1]


top3_labels_list = [encoder_fertilizer.inverse_transform(row) for row in top3]

submission = pd.DataFrame({
    'id': test['id'],
    'Fertilizer Name': [' '.join(pred_list) for pred_list in top3_labels_list]
})

submission.to_csv("submission.csv", index=False)

print("Submission file created successfully!")
print(submission.head())

Submission file created successfully!
       id          Fertilizer Name
0  750000           DAP Urea 28-28
1  750001        17-17-17 DAP Urea
2  750002      20-20 Urea 10-26-26
3  750003        Urea 17-17-17 DAP
4  750004  20-20 17-17-17 10-26-26
